In [ ]:
"""
A simple Stress test on Pandemic reovery Index using ARIMA 
Author : Babak.Ea
Date   : 2020-12-09
Target : ARIMA stress test on Recovery curves 

"""

###############################

from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import datetime
plt.style.use('ggplot')
import pandas as pd
from pandas.io.json import json_normalize 
import numpy as np
import statsmodels
import datetime
import urllib, json
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pmdarima.arima import auto_arima
import sys
import warnings
import ipywidgets as widgets
from IPython.display import display

import statsmodels.graphics.tsaplots as sgt 
import statsmodels.tsa.stattools as sts 
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
sns.set()

from math import sqrt
from sklearn.metrics import mean_squared_error



if not sys.warnoptions:
    warnings.simplefilter("ignore")


#################################
####### Database : Json #########
URL="./data/PRI_DEC_12.csv"

In [ ]:
class Data_reader:
    def __init__(self,URL,mcc_group="mcc_group",date="Date",Actual="act_cred_vol_am",predicted="prediction",
                 portfolio_type="portfolio_type",Recovery_index="Recovery_index",port_folio="National"):
        self.URL=URL
        self.Mcc=mcc_group
        
        self.date=date
        self.Actual=Actual
        self.predicted=predicted
        self.portfolio_type=portfolio_type
        self.Recovery_index=Recovery_index
        self.port_folio=port_folio

        self.DF_reader()
        #self.MA()
        
    def DF_reader(self):# input Json URL, Output: Json Data
        self.df =pd.read_csv(self.URL)
        self.df[self.date] = pd.to_datetime(self.df[self.date], infer_datetime_format=True,format='%Y-%m-%d')
        """self.df[self.date]= self.df[self.date].apply(lambda x: 
                                    dt.datetime.strptime(x,'%y%b%Y'))"""
        self.df.set_index(self.date, inplace=True)
        self.df=self.df.asfreq('b') # remove weekends
        self.df=self.df.fillna(method='ffill')# forward fill 
        
        self.mcc_group=self.df[self.Mcc].unique().tolist()
        self.df=self.df[self.df[self.portfolio_type]==self.port_folio]


        


In [ ]:
###########33
def PLOT_Df(df,col):
    plt.plot(df[col],label= 'Close')
    
    plt.plot(df[col].rolling(5).mean(),label= 'MA 5 days')
    plt.plot(df[col].rolling(7).mean(),label= 'MA 7 days')
    plt.plot(df[col].rolling(15).mean(),label= 'MA 15 days')
    plt.legend(loc='best')
    plt.title('Moneris Pandemic Recovery Index ')
    plt.show()       


In [ ]:
class RECOVERY(Data_reader):         
    def __init__(self):
        Data_reader.__init__(self,URL)
        self.ALL='ALL'
        #Country_list=self.Country_list
        
        self.dropdown_class =widgets.Dropdown(options =
                                 ['ALL','EXIT']+ sorted(self.mcc_group), description= "MCC group : " )
        self.Select_portfolio_type =widgets.Dropdown(options =
                                 ['EXIT']+ sorted(self.portfolio_type), description= "Portfolio_Type : " )
        
        self.flag="All"

        self.output = widgets.Output()
        
        self.dropdown_class.observe(self.dropdown_class_eventhandler, names='value')
        
        
        display(self.dropdown_class)
        display(self.output)
        
    def MA(self):
        self.R_5=self.Recovery_index+"_5"
        self.R_7=self.Recovery_index+"_7"
        self.R_14=self.Recovery_index+"_14"
        
        #self.A_5=self.Recovery_index+"_5"
        #self.A_7=self.Recovery_index+"_7"
        #self.A_14=self.Recovery_index+"_14"
        #tem["spx"]=tem["spx"].fillna(method='bfill')
        
        self.tem_df[self.R_5]=self.tem_df[self.Recovery_index].rolling(5).mean().fillna(method='bfill')
        self.tem_df[self.R_7]=self.tem_df[self.Recovery_index].rolling(7).mean().fillna(method='bfill')
        self.tem_df[self.R_14]=self.tem_df[self.Recovery_index].rolling(14).mean().fillna(method='bfill') 
        #print("yay")
        
        
        
        
        self.ARIMA_col=[self.Recovery_index,self.R_5,self.R_7,self.R_14]
        
    def dropdown_class_eventhandler(self,name):

        self.change=self.dropdown_class.value

        self.output.clear_output()

        with self.output:

            if (self.change== self.ALL):
                display(self.df)
                
            elif(self.change=="EXIT" ):
                print("Thnak you for using my Library \n Cheers \n Babak")
                

            else:

                self.tem_df=self.df[self.df[self.Mcc]==self.change]
                self.tem_df.set_index(self.date, inplace=True)
                self.tem_df=self.tem_df.asfreq('b')


                #print(tem.head())
                
                self.MA()
                self.tem_df=self.tem_df[self.ARIMA_col]
                
                
                #Infected=self.df[Col[0]].tolist()
                #Infected=[0]+Infected
                #self.df["Daily_infected"]=[y - x if y > x else 0 for x,y in zip(Infected,Infected[1:])]
                
                #Country_select.ARIMA_Death_Ratio(self.DF[Col])
                #ARIMA_COL=[Col[0]]+['Daily_infected']
                #Title_list=["Totla Infected in_"+ARIMA_COL[0] ,ARIMA_COL[1]+"_"+ARIMA_COL[0]]
                
                #self.COHORT_report(Col[0]) 
                """ for i in range(0,2):
                    self.ARIMA_Death_Ratio(ARIMA_COL[i],Title_list[i])"""
                for col in self.ARIMA_col:
                    self.ARIMA_RI(col,"ARIMA Forcast {}".format(col))
                  
                #self.Cohort_report(Col[0])
                #self.ARIMA_Death_Ratio()
                #self.ARIMA_Death_Ratio("Daily_infected",)
                
                #display(df)
    def ARIMA_RI(self,col,title):
        #self.tem_df=

        

        train=self.tem_df[10:-10][self.tem_df[col] != np.nan] 
        print(col)
        
        print("""
        Sectionality and auto regration:
        """)
        
        print(sts.adfuller(self.tem_df[col]))
        
        print(""" Graph Analysis : """)
        self.Ststistic_test(col)

        test=self.tem_df[-10:]
        #print(train.head())
        
        model=auto_arima(train[col], start_p=1, start_q=1,
                           max_p=30, max_q=30, m=10,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
        print(model.aic())
        model.fit(train[col])
        forcast = model.predict(n_periods=len(test[col]))

        train["Class"]="Training"
        test["Class"]="Valid Data"
        ARIMA_FORCAST=test.copy()
        ARIMA_FORCAST["Forcast"]=forcast
        
        ARIMA_FORCAST[col]=forcast
        
        DF_SNS=pd.concat([train,test,ARIMA_FORCAST])
        #del (test,train,ARIMA_FORCAST)
        DF_SNS['Day']=DF_SNS.index
        #DF_SNS['Day']=DF_SNS[self.date]
        DF_SNS['Day'] = DF_SNS['Day'].map(lambda x: x.strftime('%Y-%m-%d'))
        #DF_SNS['Day'] =  DF_SNS['Day'].astype(str)
        #DF_SNS[col] = DF_SNS[col].astype(np.float64)

        plt.figure(figsize=(13,10))
        
        ax = sns.lineplot(x="Day", y=col,
                  hue="Class", style="Class",
                  markers=True, dashes=True, data=DF_SNS)

        ax = plt.gca()

        ax.set_xticklabels(DF_SNS['Day'],rotation=45,fontsize='small')
        ax.set(xlabel="__Daily Report__", ylabel= title)

        plt.title("ARIMA Stress Test for  {}".format(title))
       
        plt.show()
        #del DF_SNS
        self.DF_SNS=DF_SNS 
        
        print(self.DF_SNS.tail())
        vall=test[col].tolist()
        ForC=list(forcast)
        rms = sqrt(mean_squared_error(vall,ForC))
        print(rms)
              
    
              
    def Ststistic_test(self,col):
        
        
        s_dec_multiplicative = seasonal_decompose(self.tem_df[col], model = "multiplicative")
        s_dec_multiplicative.plot()
        plt.figure(figsize=(13,10))
        plt.show()
        
        sgt.plot_acf(self.tem_df[col], lags = 40, zero = False)
        plt.title("ACF S&P", size = 24)
        plt.figure(figsize=(13,10))
        plt.show()
        
        sgt.plot_pacf(self.tem_df[col], lags = 40, zero = False, method = ('ols'))
        plt.title("PACF S&P", size = 24)
        plt.figure(figsize=(13,10))
        plt.show()
        
        